In [1]:
import pandas as pd
import numpy as np
import os


In [2]:
data_path = "/Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data"

files = os.listdir(data_path)

# On garde uniquement les fichiers .csv
csv_files = [f for f in files if f.endswith(".csv")]

len(csv_files), csv_files[:5]


(48, ['94 (4).csv', '95 (5).csv', '92 (1).csv', '94.csv', '78 (5).csv'])

In [4]:
df_list = []

for file in csv_files:
    file_path = os.path.join(data_path, file)
    print("Lecture de :", file_path)
    
    temp_df = pd.read_csv(
        file_path,
        sep=",",          # ✅ CORRECTION ICI
        encoding="latin1",
        low_memory=False
    )
    
    df_list.append(temp_df)

# Fusion verticale
df = pd.concat(df_list, ignore_index=True)

df.shape

Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data/94 (4).csv
Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data/95 (5).csv
Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data/92 (1).csv
Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data/94.csv
Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data/78 (5).csv
Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data/95.csv
Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data/91.csv
Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data/78 (4).csv
Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data/95 (4).csv
Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020_2025/data/77 (1).csv
Lecture de : /Users/judichamamiharilala/Documents/dvf_ile_de_france_2020

(2393734, 40)

In [5]:
# supprimer les colonnes dupliquées (problème DVF classique)
df = df.loc[:, ~df.columns.duplicated()]

df.shape


(2393734, 40)

In [6]:
colonnes_souhaitees = [
    "id_mutation",
    "date_mutation",
    "valeur_fonciere",
    "lot1_surface_carrez",   # surface
    "code_postal",
    "nom_commune",
    "code_departement"
]

# on garde seulement celles qui existent vraiment
cols_utiles = [c for c in colonnes_souhaitees if c in df.columns]

print("Colonnes gardées :", cols_utiles)

df = df[cols_utiles]
df.head()



Colonnes gardées : ['id_mutation', 'date_mutation', 'valeur_fonciere', 'lot1_surface_carrez', 'code_postal', 'nom_commune', 'code_departement']


,id_mutation,date_mutation,valeur_fonciere,lot1_surface_carrez,code_postal,nom_commune,code_departement
0,2024-1144131,2024-01-04,302000.0,NaN,94100.0,Saint-Maur-des-FossÃ©s,94
1,2024-1144131,2024-01-04,302000.0,NaN,94100.0,Saint-Maur-des-FossÃ©s,94
2,2024-1144132,2024-01-02,NaN,NaN,94500.0,Champigny-sur-Marne,94
3,2024-1144132,2024-01-02,NaN,NaN,94500.0,Champigny-sur-Marne,94
4,2024-1144132,2024-01-02,NaN,NaN,94500.0,Champigny-sur-Marne,94


In [7]:
df = df.rename(columns={
    "lot1_surface_carrez": "surface_m2",
    "nom_commune": "commune",
    "code_departement": "departement"
})

df.head()


,id_mutation,date_mutation,valeur_fonciere,surface_m2,code_postal,commune,departement
0,2024-1144131,2024-01-04,302000.0,NaN,94100.0,Saint-Maur-des-FossÃ©s,94
1,2024-1144131,2024-01-04,302000.0,NaN,94100.0,Saint-Maur-des-FossÃ©s,94
2,2024-1144132,2024-01-02,NaN,NaN,94500.0,Champigny-sur-Marne,94
3,2024-1144132,2024-01-02,NaN,NaN,94500.0,Champigny-sur-Marne,94
4,2024-1144132,2024-01-02,NaN,NaN,94500.0,Champigny-sur-Marne,94


In [8]:
df["commune"] = df["commune"].str.encode("latin1").str.decode("utf-8")
df[["commune"]].head()


,commune
0,Saint-Maur-des-Fossés
1,Saint-Maur-des-Fossés
2,Champigny-sur-Marne
3,Champigny-sur-Marne
4,Champigny-sur-Marne


In [9]:
df["date_mutation"] = pd.to_datetime(df["date_mutation"], errors="coerce")

df["valeur_fonciere"] = pd.to_numeric(df["valeur_fonciere"], errors="coerce")
df["surface_m2"] = pd.to_numeric(df["surface_m2"], errors="coerce")
df["code_postal"] = pd.to_numeric(df["code_postal"], errors="coerce")
df["departement"] = pd.to_numeric(df["departement"], errors="coerce")

df.dtypes


id_mutation                object
date_mutation      datetime64[ns]
valeur_fonciere           float64
surface_m2                float64
code_postal               float64
commune                    object
departement                 int64
dtype: object

In [10]:
df = df.dropna(subset=["date_mutation", "valeur_fonciere", "surface_m2", "code_postal"])

df.shape


(517662, 7)

In [11]:
df = df[
    (df["valeur_fonciere"] > 10_000) &   # on enlève les ventes trop petites
    (df["surface_m2"] > 9) &             # surface mini 10 m²
    (df["surface_m2"] < 500)             # on enlève les très gros biens/anomalies
]

df.shape


(508362, 7)

In [12]:
df["prix_m2"] = df["valeur_fonciere"] / df["surface_m2"]

# on enlève les prix/m² absurdes
df = df[
    (df["prix_m2"] > 500) &
    (df["prix_m2"] < 30_000)
]

df.shape, df[["valeur_fonciere", "surface_m2", "prix_m2"]].head()


((496942, 8),
     valeur_fonciere  surface_m2      prix_m2
 16         210000.0       27.53  7628.042136
 18         116600.0       23.42  4978.650726
 20         136500.0       25.34  5386.740331
 21         136500.0       25.34  5386.740331
 23         170000.0       32.52  5227.552276)

In [13]:
df["annee"] = df["date_mutation"].dt.year
df[["date_mutation", "annee"]].head()


,date_mutation,annee
16,2024-01-05,2024
18,2024-01-05,2024
20,2024-01-04,2024
21,2024-01-04,2024
23,2024-01-05,2024


In [14]:
df.to_csv("dvf_idf_clean.csv", index=False)
df.shape


(496942, 9)

In [15]:
import pandas as pd

df = pd.read_csv("dvf_idf_clean.csv")

df.head()
df.shape


(496942, 9)

In [16]:
import sqlite3

# 1. Créer / ouvrir une base SQLite
conn = sqlite3.connect("dvf_idf.db")

# 2. Envoyer le DataFrame dans une table SQL
df.to_sql("transactions_idf", conn, if_exists="replace", index=False)


496942

In [17]:
check = pd.read_sql("SELECT COUNT(*) AS nb_lignes FROM transactions_idf;", conn)
check


,nb_lignes
0,496942


In [20]:
import pandas as pd

# Quelques lignes depuis SQL
apercu = pd.read_sql("SELECT * FROM transactions_idf LIMIT 5;", conn)
display(apercu)

# Les années disponibles
annees = pd.read_sql("SELECT DISTINCT annee FROM transactions_idf ORDER BY annee;", conn)
annees


,id_mutation,date_mutation,valeur_fonciere,surface_m2,code_postal,commune,departement,prix_m2,annee
0,2024-1144134,2024-01-05,210000.0,27.53,94120.0,Fontenay-sous-Bois,94,7628.042136,2024
1,2024-1144135,2024-01-05,116600.0,23.42,94170.0,Le Perreux-sur-Marne,94,4978.650726,2024
2,2024-1144136,2024-01-04,136500.0,25.34,94100.0,Saint-Maur-des-Fossés,94,5386.740331,2024
3,2024-1144136,2024-01-04,136500.0,25.34,94100.0,Saint-Maur-des-Fossés,94,5386.740331,2024
4,2024-1144137,2024-01-05,170000.0,32.52,94100.0,Saint-Maur-des-Fossés,94,5227.552276,2024


,annee
0,2020
1,2021
2,2022
3,2023
4,2024
5,2025


In [22]:
create_query = """
CREATE TABLE IF NOT EXISTS prix_marche_commune AS
SELECT
    commune,
    departement,
    annee,
    COUNT(*) AS nb_ventes,
    AVG(prix_m2) AS prix_m2_moyen
FROM transactions_idf
GROUP BY commune, departement, annee
HAVING nb_ventes >= 20
;
"""

conn.execute("DROP TABLE IF EXISTS prix_marche_commune;")
conn.execute(create_query)
conn.commit()


In [23]:
apercu_marche = pd.read_sql(
    "SELECT * FROM prix_marche_commune ORDER BY departement, commune, annee LIMIT 10;",
    conn
)
apercu_marche


,commune,departement,annee,nb_ventes,prix_m2_moyen
0,Paris 10e Arrondissement,75,2020,498,10734.427234
1,Paris 10e Arrondissement,75,2021,1726,10637.130723
2,Paris 10e Arrondissement,75,2022,1894,10463.048486
3,Paris 10e Arrondissement,75,2023,1283,10078.105278
4,Paris 10e Arrondissement,75,2024,1142,9427.962438
5,Paris 10e Arrondissement,75,2025,599,9324.503845
6,Paris 11e Arrondissement,75,2020,813,11061.706060
7,Paris 11e Arrondissement,75,2021,2270,11213.546676
8,Paris 11e Arrondissement,75,2022,2912,10966.621682
9,Paris 11e Arrondissement,75,2023,2429,10339.382038


In [24]:
pd.read_sql("SELECT COUNT(*) AS nb_lignes FROM prix_marche_commune;", conn)


,nb_lignes
0,2183


In [25]:
create_eval_query = """
DROP TABLE IF EXISTS transactions_eval;

CREATE TABLE transactions_eval AS
SELECT
    t.id_mutation,
    t.date_mutation,
    t.valeur_fonciere,
    t.surface_m2,
    t.code_postal,
    t.commune,
    t.departement,
    t.prix_m2,
    t.annee,
    p.prix_m2_moyen,
    ROUND( (t.prix_m2 - p.prix_m2_moyen) / p.prix_m2_moyen * 100, 2 ) AS ecart_pct,
    CASE
        WHEN (t.prix_m2 - p.prix_m2_moyen) / p.prix_m2_moyen * 100 <= -10 THEN 'Sous-évalué'
        WHEN (t.prix_m2 - p.prix_m2_moyen) / p.prix_m2_moyen * 100 >= 10 THEN 'Sur-évalué'
        ELSE 'Prix cohérent'
    END AS statut_eval
FROM transactions_idf t
JOIN prix_marche_commune p
    ON t.commune = p.commune
   AND t.departement = p.departement
   AND t.annee = p.annee;
"""

conn.executescript(create_eval_query)
conn.commit()


In [26]:
apercu_eval = pd.read_sql(
    "SELECT * FROM transactions_eval LIMIT 10;",
    conn
)
apercu_eval


,id_mutation,date_mutation,valeur_fonciere,surface_m2,code_postal,commune,departement,prix_m2,annee,prix_m2_moyen,ecart_pct,statut_eval
0,2024-1144134,2024-01-05,210000.0,27.53,94120.0,Fontenay-sous-Bois,94,7628.042136,2024,5685.392063,34.17,Sur-évalué
1,2024-1144135,2024-01-05,116600.0,23.42,94170.0,Le Perreux-sur-Marne,94,4978.650726,2024,5512.121234,-9.68,Prix cohérent
2,2024-1144136,2024-01-04,136500.0,25.34,94100.0,Saint-Maur-des-Fossés,94,5386.740331,2024,5837.603815,-7.72,Prix cohérent
3,2024-1144136,2024-01-04,136500.0,25.34,94100.0,Saint-Maur-des-Fossés,94,5386.740331,2024,5837.603815,-7.72,Prix cohérent
4,2024-1144137,2024-01-05,170000.0,32.52,94100.0,Saint-Maur-des-Fossés,94,5227.552276,2024,5837.603815,-10.45,Sous-évalué
5,2024-1144140,2024-01-03,465000.0,75.14,94160.0,Saint-Mandé,94,6188.448230,2024,8824.614823,-29.87,Sous-évalué
6,2024-1144140,2024-01-03,465000.0,75.14,94160.0,Saint-Mandé,94,6188.448230,2024,8824.614823,-29.87,Sous-évalué
7,2024-1144143,2024-01-03,555000.0,53.00,94300.0,Vincennes,94,10471.698113,2024,8677.412062,20.68,Sur-évalué
8,2024-1144144,2024-01-08,285000.0,48.14,94700.0,Maisons-Alfort,94,5920.232655,2024,5520.027099,7.25,Prix cohérent
9,2024-1144144,2024-01-08,285000.0,48.14,94700.0,Maisons-Alfort,94,5920.232655,2024,5520.027099,7.25,Prix cohérent


In [27]:
pd.read_sql("""
SELECT statut_eval, COUNT(*) AS nb
FROM transactions_eval
GROUP BY statut_eval;
""", conn)


,statut_eval,nb
0,Prix cohérent,174232
1,Sous-évalué,184945
2,Sur-évalué,124811


In [28]:
df_eval = pd.read_sql("SELECT * FROM transactions_eval;", conn)

df_eval.to_csv("transactions_eval.csv", index=False)

df_eval.head(), df_eval.shape


(    id_mutation date_mutation  valeur_fonciere  surface_m2  code_postal  \
 0  2024-1144134    2024-01-05         210000.0       27.53      94120.0   
 1  2024-1144135    2024-01-05         116600.0       23.42      94170.0   
 2  2024-1144136    2024-01-04         136500.0       25.34      94100.0   
 3  2024-1144136    2024-01-04         136500.0       25.34      94100.0   
 4  2024-1144137    2024-01-05         170000.0       32.52      94100.0   
 
                  commune  departement      prix_m2  annee  prix_m2_moyen  \
 0     Fontenay-sous-Bois           94  7628.042136   2024    5685.392063   
 1   Le Perreux-sur-Marne           94  4978.650726   2024    5512.121234   
 2  Saint-Maur-des-Fossés           94  5386.740331   2024    5837.603815   
 3  Saint-Maur-des-Fossés           94  5386.740331   2024    5837.603815   
 4  Saint-Maur-des-Fossés           94  5227.552276   2024    5837.603815   
 
    ecart_pct    statut_eval  
 0      34.17     Sur-évalué  
 1      -9.68  P

,nb_lignes
0,496942
